In [26]:
import os
import json
import random
import pandas as pd
import importlib

import config
import AbstractModel
import gpt
import prompts as promptlib

importlib.reload(config)
importlib.reload(AbstractModel)
importlib.reload(gpt)
importlib.reload(promptlib)

config.source_dot_env()

SAMPLE_SIZE = 20
data = {
    "skill": [],
    "prompt": [],
    "tone": [],
    "errors": [],
}
for i in range(SAMPLE_SIZE):
    skill = random.choice(list(promptlib.SKILLS.keys()))
    tone = random.choice(promptlib.TONES)
    skill_description = promptlib.SKILLS[skill]
    
    extra = ""
    smart_errors = []
    if random.randint(0, 1) == 0:
        # randomly make a few errors in SMART formulation
        num_errors = random.choices([1, 2, 3], weights=[0.5, 0.35, 0.15])[0]
        smart_errors = random.sample(promptlib.SMART, num_errors)
        extra = f"Also intentionally formulate your SMART goal/plan such that the following attributes are not adherent to the SMART formulation: {', '.join(smart_errors)}."

    prompt = promptlib.PROMPT_SYTHNETIC_SMART.format(skill=skill, skill_description=skill_description, tone=tone, extra=extra)
    data["skill"].append(skill)
    data["tone"].append(tone)
    data["prompt"].append(prompt)
    smart_errors.sort(key=lambda x: promptlib.SMART.index(x))
    data["errors"].append(smart_errors)


df = pd.DataFrame(data)
# sort by len(errors) and reindex
df = df.sort_values(by="errors").reset_index(drop=True)

display(df)

,skill,prompt,tone,errors
0,voice volume,You're a student taking a course where you're ...,respectful,[]
1,language use,You're a student taking a course where you're ...,authoritative,[]
2,facial expression: ability to use facial expre...,You're a student taking a course where you're ...,serious,[]
3,language use,You're a student taking a course where you're ...,informal,[]
4,language use,You're a student taking a course where you're ...,formal,[]
5,facial expression: ability to use facial expre...,You're a student taking a course where you're ...,motivating,[]
6,facial expression: ability to use facial expre...,You're a student taking a course where you're ...,optimistic,[]
7,clarity in communication,You're a student taking a course where you're ...,motivating,[]
8,facial expression: ability to use facial expre...,You're a student taking a course where you're ...,humorous,[]
9,hand gestures,You're a student taking a course where you're ...,formal,[action-oriented]


In [31]:
model = gpt.GPTModel() # gpt 3
outputs, meta = model(list(df["prompt"]), json_mode=True)
print(f"price = ${model.compute_price(meta):.3f}")

price = $0.011050999999999998


In [32]:
smart, plan = [], []
for output in outputs:
    obj = json.loads(output)
    smart.append(obj["smart"])
    plan.append(obj["plan"])

print(smart)
print(plan)
df = df.assign(smart=smart, plan=plan)

price = $0.011050999999999998
['Increase my voice volume during presentations in order to ensure that everyone in the audience can hear me clearly. I will achieve this by practicing speaking louder during rehearsals, recording myself to monitor progress, and seeking feedback from peers on the clarity and volume of my voice.', 'Increase my proficiency in using precise scientific terminology and professional language in my written assignments for the course.', 'Demonstrate active listening skills during group discussions by maintaining eye contact, nodding in agreement, and asking clarifying questions to show engagement.', 'Improve my ability to use precise and professional language in my written assignments by reducing the use of filler words and ensuring the appropriate use of scientific terminology.', 'To improve my ability to use precise and professional language in my assignments, ensuring the appropriate use of scientific terminology and avoiding filler words.', 'Improve my time ma

In [33]:
fname = "synthetic_smart_v2.csv"

df.to_csv(fname, index=False)
print(f"saved to '{fname}'")

saved to 'synthetic_smart_v2.csv'
